In [0]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform
import os

In [0]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [0]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"

# OPENAI_MODEL = "gpt-4o"
# CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [0]:
processor = platform.machine()

In [0]:
def get_name_by_extension(extension):
    for lang in programming_languages:
        if lang["extension"] == extension:
            return lang["name"]
    return None 

In [0]:
def get_system_message(prog_lang):
    name = get_name_by_extension(prog_lang)
    
    system_message = f"You are an assistant that reimplements Python code to {name} for an {processor} device. "
    system_message += f"Respond only with code; use comments sparingly and do not provide any explanation other than occasional comments. "
    system_message += f"The {name} response needs to produce an identical output in the fastest possible time."
    system_message += f"If the used function does not exists for {name} language interchange it for its compatibility and if not throw an error"

    return system_message

In [0]:
def user_prompt_for(python, prog_lang):
    name = get_name_by_extension(prog_lang)
    
    user_prompt = f"Rewrite this Python code in {name} with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with {name} code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows\n\n"
    user_prompt += python
    return user_prompt

In [0]:
def messages_for(python, prog_lang):
    system_message = get_system_message(prog_lang)
    
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python, prog_lang)}
    ]

In [0]:
# write to a file called optimized.cpp

def write_output(content, prog_lang):
    code = content.replace("```cpp","").replace("javascript","").replace("```","")
    
    with open(f"optimized.{prog_lang}", "w") as f:
        f.write(code)

In [0]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [0]:
def stream_gpt(python, prog_lang):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python, prog_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('javascript\n','').replace('```','')

In [0]:
def stream_claude(python, prog_lang):
    system_message = get_system_message(prog_lang)
    
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python, prog_lang)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('javascript\n','').replace('```','')

In [0]:
def optimize(python, model, prog_lang):
    if model=="GPT":
        result = stream_gpt(python, prog_lang)
    elif model=="Claude":
        result = stream_claude(python, prog_lang)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [0]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output

        namespace = {}
        exec(code, namespace)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [0]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.php {background-color: #cb7afa;}
.js {background-color: #f4ff78;}
"""

In [0]:
def execute_cpp(code):
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, shell=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [0]:
def execute_js(code):
        try:
            run_result = subprocess.run(["node", "optimized.js"], check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [0]:
def execute_php(code):
        try:
            run_result = subprocess.run(["php", "optimized.php"], check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"


In [0]:
def handle_execution(code, prog_lang):
    write_output(code, prog_lang)

    index = next((i for i, lang in enumerate(programming_languages) if lang["extension"] == prog_lang), -1)
    return programming_languages[index]["fn"](code)

In [0]:
programming_languages = [
    {"name": "C++", "extension": "cpp", "fn": execute_cpp},
    {"name": "Javascript", "extension": "js", "fn": execute_js},
    {"name": "Php", "extension": "php", "fn": execute_php}
]

In [0]:
def create_prog_lang_ui(lang, model):
    prog_name = lang["name"]
    extension = lang["extension"]
    fn = lang["fn"]

    with gr.Row():
        with gr.Column():
            convert = gr.Button(f"Convert to {prog_name}")
            converted_code = gr.Textbox(label=f"Converted {prog_name} code:", lines=10)

        with gr.Column():
            prog_run = gr.Button(f"Run {prog_name}")
            prog_out = gr.TextArea(label=f"{prog_name} result:", elem_classes=[extension])

    current_selected = gr.Dropdown([extension], value=extension, visible=False)
            
    convert.click(optimize, inputs=[python, model, current_selected], outputs=[converted_code])
    prog_run.click(handle_execution, inputs=[converted_code, current_selected], outputs=[prog_out])

with gr.Blocks(css=css) as ui:
    gr.Markdown("# Convert code from Python to any Programming Language")
    with gr.Row():
        with gr.Column():
            python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        with gr.Column():
            python_run = gr.Button(f"Run Python")
            python_out = gr.TextArea(label=f"Python result:", elem_classes=["python"])
            
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")

    python_run.click(execute_python, inputs=[python], outputs=[python_out])  


    for lang in programming_languages:
        create_prog_lang_ui(lang, model)

ui.launch(
    inbrowser=True
)